# Airbnb princing

In Airbnb the pricing should be set by the owner of the property. Their suggestion is that the rate should be based on the monthly mortgage. 

In [1]:
import pandas as pd
import numpy as np

raw_data = pd.read_csv('../data/Ames_Housing_Price_Data.csv', index_col=0)

rental_df = raw_data[['PID','SalePrice','YrSold']]
rental_df.sample(10)

,PID,SalePrice,YrSold
470,535353160,138000,2007
430,908203100,90000,2010
269,908102100,140000,2007
86,903227090,132500,2009
710,534128100,162000,2009
711,913350030,242500,2006
123,528240130,185088,2010
251,527378020,157000,2008
425,910205130,152000,2007
910,527368020,216000,2010


Group by OverallQual and get the mean and median SalePrice for each category

In [3]:
rental_df = pd.merge(
    rental_df,
    mortgage_df,
    how="left",
    left_on='YrSold',
    right_on='Year',
    sort=True,
    suffixes=("_x", "_y"),
    copy=False).drop(columns = ['Year'])

rental_df.sample(10)

,PID,SalePrice,YrSold,Average_30Year_Rate
2355,902206220,129000,2010,4.69
847,905478220,136500,2007,6.34
728,905426010,243000,2007,6.34
759,527451170,122500,2007,6.34
2244,907135040,145000,2009,5.04
1623,531450090,179200,2008,6.03
777,535101150,126000,2007,6.34
2292,534202160,142250,2010,4.69
745,533206060,166000,2007,6.34
2493,534475100,128000,2010,4.69


In [4]:
rental_df['YrSold'].value_counts()

2007    600
2009    585
2008    564
2006    516
2010    315
Name: YrSold, dtype: int64

According to Forbes, 2006 downpayment was 20%

Calculate monthly mortgage payment, not including taxes and insurance, using the following equation:

M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]

P = 0.8*SalePrice <- principal loan amount

i = monthly interest rate

n = 30*12 = 360

In [2]:
mortgage_df = pd.read_csv('data/MortageRates.csv')
mortgage_df['Average_30Year_Rate']= mortgage_df['Average_30Year_Rate'].str.replace('%','').astype(float)
mortgage_df.sample(10)

,Year,Average_30Year_Rate
6,1978,9.64
13,1985,12.43
19,1991,9.25
22,1994,8.38
25,1997,7.60
18,1990,10.13
40,2012,3.66
11,1983,13.24
16,1988,10.34
9,1981,16.63


In [5]:
P = 0.8*rental_df['SalePrice']
i = rental_df['Average_30Year_Rate'].map(lambda x: x/(100*12))
n = 360
rental_df['MMortgage'] = P*((i*(1+i)**n))/(((1+i)**n)-1)
rental_df.sample(10)

rental_df['MMor_utilities'] = rental_df['MMortgage'] + 215 #adding average monthly utilities
rental_df['day_rate'] = (rental_df['MMor_utilities']/30).round(2)

rental_df.sample(10)

,PID,SalePrice,YrSold,Average_30Year_Rate,MMortgage,MMor_utilities,day_rate
242,905452140,136900,2006,6.41,685.771331,900.771331,30.03
2273,907200340,128200,2010,4.69,531.298671,746.298671,24.88
2138,906392130,173000,2009,5.04,746.348171,961.348171,32.04
2313,902207130,12789,2010,4.69,53.001394,268.001394,8.93
774,902205130,108500,2007,6.34,539.533643,754.533643,25.15
1574,905452070,99600,2008,6.03,479.259776,694.259776,23.14
1868,902207170,99500,2009,5.04,429.258052,644.258052,21.48
258,533212120,149900,2006,6.41,750.892057,965.892057,32.20
358,908188140,137500,2006,6.41,688.776903,903.776903,30.13
2258,903202170,137000,2009,5.04,591.038725,806.038725,26.87


monthly utilities sourcehttp://www.spacesimply.com/article.aspx?id=20&title=The+Price+We+Pay+to+Live+in+Iowa+(Infographic)

Do the calculations shown above with the Sale Prices as predicted by our gradient boost model

In [6]:
pred = pd.read_csv('data/Boosted_predictions.csv')
pred = pd.DataFrame(pred)
pred.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,YrSold,SaleType,SaleCondition,lat,lon,dist_stadium,dist_downtown,dist_airport,Price_sqft,Predicted
0,909176150,856,126000,30,RL,58.769231,7890,Pave,DNE,Reg,...,2010,WD,Normal,42.018564,-93.651619,0.835068,1.997560,2.024702,147.196262,125221.178278
1,905476230,1049,139500,120,RL,42.000000,4235,Pave,DNE,Reg,...,2009,WD,Normal,42.024855,-93.663671,1.558185,2.566078,2.777751,132.983794,133876.412024
2,911128020,1001,124900,30,C (all),60.000000,6060,Pave,DNE,Reg,...,2007,WD,Normal,42.021548,-93.614068,1.173123,0.245745,1.616918,124.775225,119262.951886
3,535377150,1039,114000,70,RL,80.000000,8146,Pave,DNE,Reg,...,2009,WD,Normal,42.037391,-93.612207,1.897608,0.852123,2.704025,109.720885,106346.649449
4,534177230,1665,227000,60,RL,70.000000,8400,Pave,DNE,Reg,...,2009,WD,Normal,42.044554,-93.631818,1.966935,1.631256,3.184678,136.336336,209245.026345


In [7]:
airbnb_rental_df = pd.merge(
    pred,
    mortgage_df,
    how="inner",
    left_on='YrSold',
    right_on='Year',
    sort=True,
    suffixes=("_x", "_y"),
    copy=False).drop(columns = ['Year'])

airbnb_rental_df.shape

(2568, 89)

In [8]:
P = 0.8*airbnb_rental_df['Predicted']
i = airbnb_rental_df['Average_30Year_Rate'].map(lambda x: x/(100*12))
n = 360
airbnb_rental_df['MMortgage'] = P*((i*(1+i)**n))/(((1+i)**n)-1)

airbnb_rental_df['MMor_utilities'] = airbnb_rental_df['MMortgage'] + 215 #adding average monthly utilities
airbnb_rental_df['day_rate'] = (airbnb_rental_df['MMor_utilities']/30).round(2)
airbnb_rental_df['weekend_rate'] = airbnb_rental_df['day_rate'] *2
airbnb_rental_df.shape

(2568, 93)

# Maps

Make maps to assist with data visualization

In [42]:
import plotly.express as px

fig = px.scatter_mapbox(airbnb_rental_df, lat="lat", lon="lon", hover_name="PID", #all the houses in the dataset
                        hover_data=['weekend_rate'], color='weekend_rate', zoom=12,
                        height=500, color_continuous_scale = 'turbo')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Limit the visualization to the houses that meet Airbnb standards

In [11]:
airbnb_houses = pd.read_csv('data/airbnb_houses.csv', index_col=False)

airbnb_houses.head()

,Unnamed: 0,PID,SalePrice,GrLivArea,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YrSinceUpdate,...,Fence,MiscFeature,Neighborhood,lat,lon,dist_stadium,dist_downtown,dist_airport,coord,to_stadium
0,0,905476230,139500,1049,TwnhsE,1Story,5,5,1984,26,...,None,None,Edwards,42.024855,-93.663671,1.558185,2.566078,2.777751,"(42.024855315930424, -93.66367099341755)",biking
1,1,905476230,139500,1049,TwnhsE,1Story,5,5,1984,26,...,None,None,Edwards,42.024855,-93.663671,1.558185,2.566078,2.777751,"(42.024855315930424, -93.66367099341755)",biking
2,2,908128060,198500,1922,1Fam,SFoyer,7,5,2003,7,...,None,None,Edwards,42.019009,-93.672987,1.928481,3.074472,2.950477,"(42.01900877172312, -93.67298690279817)",biking
3,3,923252075,131250,1091,1Fam,SLvl,6,5,1993,14,...,None,Shed,Mitchel,41.987528,-93.606181,2.495401,2.622904,1.147281,"(41.98752773737374, -93.60618059595959)",biking
4,4,902201120,115000,789,1Fam,1Story,5,5,1948,60,...,None,None,OldTown,42.033518,-93.608836,1.827655,0.634668,2.485242,"(42.03351832653061, -93.6088355510204)",biking


In [12]:
airbnb_houses_rental = pd.merge(airbnb_houses, airbnb_rental_df, how = 'left', on = "PID")

In [36]:
fig = px.scatter_mapbox(airbnb_houses_rental, lat="lat_y", lon="lon_y", hover_name="PID",
                        color='weekend_rate', zoom=12, height=500, 
                        color_continuous_scale = 'turbo')#only the houses that meet airbnb stds
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

How much will Airbnb make? They charge the hosts 3% and the guests 14.2 % of the booking subtotal

https://www.airbnb.com/help/article/1857/what-are-airbnb-service-fees

In [14]:
weekend_profit_max = ((airbnb_houses_rental["weekend_rate"].sum())*0.17).round(2)
weekend_profit_max

2129.35

In [15]:
year_profit_max = ((airbnb_houses_rental["day_rate"].sum())*360*0.17).round(2)
year_profit_max

383283.36

Map to show how many guests each house can host. The numbers are estimating that there can be 2 guests hosted per room

In [16]:
airbnb_houses_rental['BedroomAbvGr_x']

0      2
1      2
2      4
3      2
4      2
      ..
175    3
176    4
177    3
178    3
179    4
Name: BedroomAbvGr_x, Length: 180, dtype: int64

In [17]:
conditions = [
    (airbnb_houses_rental['BedroomAbvGr_x'] >= 0) & (airbnb_houses_rental['BedroomAbvGr_x'] <=1),
    (airbnb_houses_rental['BedroomAbvGr_x'] == 2),
    (airbnb_houses_rental['BedroomAbvGr_x'] >=3)]
choices = ['2', '4', '6+']
airbnb_houses_rental['guests'] = np.select(conditions, choices, default='black')
airbnb_houses_rental

,Unnamed: 0,PID,SalePrice_x,GrLivArea_x,BldgType_x,HouseStyle_x,OverallQual_x,OverallCond_x,YearBuilt_x,YrSinceUpdate,...,dist_downtown_y,dist_airport_y,Price_sqft,Predicted,Average_30Year_Rate,MMortgage,MMor_utilities,day_rate,weekend_rate,guests
0,0,905476230,139500,1049,TwnhsE,1Story,5,5,1984,26,...,2.566078,2.777751,132.983794,133876.412024,5.04,577.563094,792.563094,26.42,52.84,4
1,1,905476230,139500,1049,TwnhsE,1Story,5,5,1984,26,...,2.566078,2.777751,132.983794,133876.412024,5.04,577.563094,792.563094,26.42,52.84,4
2,2,908128060,198500,1922,1Fam,SFoyer,7,5,2003,7,...,3.074472,2.950477,103.277836,201015.800851,5.04,867.212574,1082.212574,36.07,72.14,6+
3,3,923252075,131250,1091,1Fam,SLvl,6,5,1993,14,...,2.622904,1.147281,120.302475,137296.457569,5.04,592.317688,807.317688,26.91,53.82,4
4,4,902201120,115000,789,1Fam,1Story,5,5,1948,60,...,0.634668,2.485242,145.754119,100351.500008,6.03,482.875877,697.875877,23.26,46.52,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,175,906420130,228000,2097,1Fam,2Story,7,5,1993,17,...,3.527576,3.759553,108.726753,246105.653338,6.03,1184.222291,1399.222291,46.64,93.28,6+
176,176,905476120,119000,1367,1Fam,1.5Fin,5,8,1900,60,...,2.735033,2.789998,87.051939,121763.520369,6.03,585.907203,800.907203,26.70,53.40,6+
177,177,905105200,137900,892,1Fam,1Story,5,7,1966,2,...,3.318132,3.573622,154.596413,131432.555348,4.69,544.695336,759.695336,25.32,50.64,6+
178,178,534129370,163000,1336,1Fam,1Story,5,5,1969,41,...,1.716878,3.266308,122.005988,161165.310384,5.04,695.291530,910.291530,30.34,60.68,6+


In [18]:
fig = px.scatter_mapbox(airbnb_houses_rental, lat="lat_y", lon="lon_y", hover_name="PID",
                        color='guests', zoom=12, height=500, color_continuous_scale = 'plasma')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

General maps

In [9]:
full_latlon = pd.read_csv('data/full_latlon.csv')
full_latlon['type']='house'
full_latlon.sample(3)

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,dist_stadium,dist_downtown,dist_airport,Price_sqft,Predicted,Average_30Year_Rate,MMortgage,MMor_utilities,day_rate,weekend_rate


In [ ]:
airbnb_houses = pd.read_csv('data/airbnb_houses.csv', index_col=False)
airbnb_houses['type']= 'airbnb_house'
airbnb_houses.shape

In [44]:
airbnb_houses.loc[airbnb_houses['PID'] == 909428120, 'to_stadium'] = 'walking' #this house is really close to the
                                                                                #stadium but on the other side of 
                                                                                #the main entrance

In [45]:
fig = px.scatter_mapbox(airbnb_houses, lat="lat", lon="lon", hover_name="PID", hover_data=["to_stadium"],
                        color="to_stadium",size = 'SalePrice', zoom=12, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [41]:
fig = px.scatter_mapbox(airbnb_houses, lat="lat", lon="lon", hover_name="PID", hover_data=["dist_stadium"],
                        color="Neighborhood",size = 'SalePrice', zoom=12, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [8]:
landmarks = pd.DataFrame({'type': ['airport' , 'stadium','downtown'],
                          'lat':[41.9989524,42.0162027, 42.0251029],
                          'lon':[-93.6223669, -93.6357060, -93.6138075]})

landmarks

,type,lat,lon
0,airport,41.998952,-93.622367
1,stadium,42.016203,-93.635706
2,downtown,42.025103,-93.613807


In [68]:
full_and_airbnb = pd.concat([airbnb_houses, full_latlon,landmarks])
full_and_airbnb = full_and_airbnb.sort_values(by='type')


In [105]:

fig = px.scatter_mapbox(full_and_airbnb, lat="lat", lon="lon",
                        color="type", zoom=12, opacity = 1, 
                        height=500,category_orders  ={"type":['house', 'airbnb_house'] },
                        color_discrete_sequence = (['#EFAA52','#AB3E16', '#351F39', 'Darkblue', 'DarkGreen']))
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
